In [1]:
using Pkg
# Pkg.activate(joinpath(@__DIR__, "../../"))
using Globtim
using DynamicPolynomials, DataFrames
using ProgressLogging
using StaticArrays

We set the number of samples used to generate the approximant. It is annoying that the error goes up while the degree has increased.

In [4]:
# Choose f from: Rastringin, alpine1, Ackley
f = Ackley

# Choose n from: 2, 3, 4
n = 2

# Constants and Parameters
const a, b = 1, 1
const scale_factor = a / b
rand_center = zeros(Float64, n);
d = 10 # initial degree 
SMPL = 40 # Number of samples
TR = test_input(f, 
                dim = n,
                center=rand_center,
                GN=SMPL, 
                sample_range=scale_factor 
                )

test_input(2, [0.0, 0.0], 40, (0.1, 0.5), 0.002, (0.0, 0.0), 1.0, 1.0, 6, Globtim.Ackley)

Solve the system of partial derivatives using `Homotopy_COntinuation.jl`. 

In [5]:
pol_cheb = Constructor(TR, d, basis=:chebyshev, precision=RationalPrecision)
@polyvar(x[1:n]); # Define polynomial ring 

current L2-norm: 0.1700717268466939


In [6]:
@polyvar(x[1:n]) # Define polynomial ring 
real_pts_cheb = solve_polynomial_system(
    x, n, d, pol_cheb.coeffs;
    basis=pol_cheb.basis,
    precision=pol_cheb.precision,
    normalized=pol_cheb.normalized,
)

Tracking 81 paths...   2%|▊                             |  ETA: 0:06:17



Tracking 81 paths... 100%|██████████████████████████████| Time: 0:00:10
                   # paths tracked: 81
   # non-singular solutions (real): 81 (17)
       # singular endpoints (real): 0 (0)
          # total solutions (real): 81 (17)


17-element Vector{Vector{Float64}}:
 [0.9640968143739056, 0.9640968143739062]
 [6.330804320179573e-18, 0.9629131980007141]
 [-0.9640968143739098, 0.9640968143739023]
 [-0.6981226790435427, 0.6981226790435454]
 [0.9629131980007143, 1.5007229786735243e-16]
 [2.068902094908175e-17, 2.7491256920457325e-16]
 [-0.9629131980007145, 2.1092288166840942e-16]
 [-0.6528936546007026, 3.20712876174557e-16]
 [0.6528936546007027, 3.388980016893826e-16]
 [0.698122679043523, -0.698122679043534]
 [0.9640968143739028, -0.9640968143739072]
 [3.2096778590629643e-17, -0.9629131980007158]
 [-0.9640968143739042, -0.9640968143739079]
 [-0.6981226790435237, -0.6981226790435394]
 [3.422725992320806e-17, -0.6528936546006976]
 [2.7162092840535903e-17, 0.6528936546007063]
 [0.698122679043538, 0.698122679043529]

In [7]:
df_cheb = process_crit_pts(real_pts_cheb, f, TR)

Row,x1,x2,z
,Float64,Float64,Float64
1,0.964097,0.964097,3.57539
2,6.3308e-18,0.962913,2.58271
3,-0.964097,0.964097,3.57539
4,-0.698123,0.698123,4.59859
5,0.962913,1.50072e-16,2.58271
6,2.0689e-17,2.74913e-16,4.44089e-16
7,-0.962913,2.10923e-16,2.58271
8,-0.652894,3.20713e-16,3.24424
9,0.652894,3.38898e-16,3.24424


In [6]:
using Optim
df_cheb, df_min_cheb = analyze_critical_points(f, df_cheb, TR, tol_dist=0.5);

Processing point 1 of 125
Optimization has converged within bounds: ✓
Processing point 2 of 125
Optimization has converged within bounds: ✓
Processing point 3 of 125
Optimization has converged within bounds: ✓
Processing point 4 of 125
Optimization has converged within bounds: ✓
Processing point 5 of 125
Optimization has converged within bounds: ✓
Processing point 6 of 125
Optimization has converged within bounds: ✓
Processing point 7 of 125
Optimization has converged within bounds: ✓
Processing point 8 of 125
Optimization has converged within bounds: ✓
Processing point 9 of 125
Optimization has converged within bounds: ✓
Processing point 10 of 125
Optimization has converged within bounds: ✓
Processing point 11 of 125
Optimization has converged within bounds: ✓
Processing point 12 of 125
Optimization has converged within bounds: ✓
Processing point 13 of 125
Optimization has converged within bounds: ✓
Processing point 14 of 125
Optimization has converged within bounds: ✓
Processing poin

In [13]:
df_cheb

Row,x1,x2,z
,Float64,Float64,Float64
1,0.964097,0.964097,3.57539
2,6.3308e-18,0.962913,2.58271
3,-0.964097,0.964097,3.57539
4,-0.698123,0.698123,4.59859
5,0.962913,1.50072e-16,2.58271
6,2.0689e-17,2.74913e-16,4.44089e-16
7,-0.962913,2.10923e-16,2.58271
8,-0.652894,3.20713e-16,3.24424
9,0.652894,3.38898e-16,3.24424
